# Display Guild Lineage Tables

In [1]:
import os, sys
import pandas as PD

In [2]:
BACKEND_PATH = os.path.dirname(os.path.dirname(os.getcwd()))
APPPROJ_PATH = os.path.join(BACKEND_PATH, 'app_proj')
sys.path.append(APPPROJ_PATH)
import notebooks as NT
print([x for x in dir(NT) if x[0]!='_' and len(x)>8])

['DataframeToDicts', 'GetRandom', 'NOTEBOOK_ENV']


In [3]:
MODULE_PATH = os.path.dirname(os.getcwd())
DATA_PATH = os.path.join(MODULE_PATH, 'data')
os.path.exists(DATA_PATH)

True

In [4]:
import django
os.environ['DEPLOYMENT_ENV'] = NT.NOTEBOOK_ENV
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'True'
os.chdir(os.path.dirname(MODULE_PATH))
django.setup()

env: DEV


In [5]:
import django.db.models as JM
from django.contrib.auth import get_user_model

In [6]:
import database as DB
print([x for x in dir(DB) if x[0]!='_' and len(x)>8])

['BaseManager', 'DeleteTable', 'GetNativeTableCounts', 'GetTableCounts', 'GetTableDictionary', 'InsertBulk', 'InsertSingle']


In [7]:
import engine.models as GM
import engine.logic.resource as RS
import engine.logic.content as CT
import engine.logic.launcher as LH
import engine.logic.robot as RB
print([x for x in dir(GM) if x[0]!='_' and len(x)>8])

['CommonWares', 'GuildAssets', 'GuildRamp', 'GuildTower', 'ItemInGuild', 'ItemUnlocked', 'MagicMarket', 'ThiefInGuild', 'ThiefUnlocked']


In [8]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

### Guild Tables

In [9]:
userTb = get_user_model()
adminMd = userTb.objects.getUser(email='admin@thieves-guild.com')
adminMd

<User: admin@thieves-guild.com>

In [10]:
guildMds = GM.Guild.objects.filter(UserFK=adminMd)
guildLs = list(guildMds.values())

if len(guildLs) > 0:
    guildDf = PD.DataFrame(guildLs)
    guildDf = guildDf.drop(['UserFK_id', 'VaultGold', 'VaultGems'], axis=1, errors='ignore')
    guildDf = guildDf.sort_values('LastPlayed', ascending=False)
    guildDf
else:
    print('no guilds')

,id,Name,KeepLevel,TotalPower,Selected,LastPlayed,CreateDate
0,29,123,1,576,True,2023-12-18,2023-12-18


In [11]:
guildName = '123'
guildMd = GM.Guild.objects.GetOrNone(Name=guildName)
thiefMds = GM.ThiefInGuild.objects.filter(GuildFK=guildMd)
thiefLs = list(thiefMds.values())

thiefDf = PD.DataFrame(thiefLs)
thiefDf = thiefDf.drop(['GuildFK_id', 'BaseAgi', 'BaseCun', 'BaseMig', 'BaseEnd', 
                        'TrainedAgi', 'TrainedCun', 'TrainedMig', 'TrainedEnd',
                        'Experience', 'Position', 'Wounds', 'Cooldown'], 
                       axis=1, errors='ignore')
thiefDf = thiefDf.sort_values('Power', ascending=False)
thiefDf

,id,Name,Class,Stars,BasePower,Level,Power,Agility,Cunning,Might,Endurance,Health,Attack,Damage,Defense,Sabotage,Perceive,Traverse
0,146,Dresden,Burglar,1,48,1,96,5,0,0,0,58,5,8,10,0,0,0
1,147,Doomington,Scoundrel,1,48,1,96,0,5,0,0,58,4,11,10,0,0,0
2,148,Kaore,Ruffian,1,48,1,96,0,0,5,0,58,6,6,15,0,0,0
3,149,Smetrios,Burglar,1,48,1,96,5,0,0,0,58,5,8,10,0,0,0
4,150,Morgoz,Scoundrel,1,48,1,96,0,5,0,0,58,4,11,10,0,0,0
5,151,Daville,Ruffian,1,48,1,96,0,0,5,0,58,6,6,15,0,0,0


In [12]:
assetMds = GM.ItemInGuild.objects.filter(GuildFK=guildMd)
assetLs = list(assetMds.values())

assetDf = PD.DataFrame(assetLs)
assetDf = assetDf.drop(['GuildFK_id', ], axis=1, errors='ignore')
assetDf = assetDf.sort_values('Power', ascending=False)
assetDf

,id,ThiefFK_id,Name,Level,Slot,Power,Requirement,Trait,Combat,Skill,Magic
0,433,146.0,Blackthorn,1,weapon,24,Burglar,agi 1,dmg 1,None,None
7,440,149.0,Brigandine,1,armor,24,Burglar,agi 1,dmg 1,None,None
11,444,151.0,Chain Maille,1,armor,24,Ruffian,mig 1,att 3,None,None
10,443,151.0,Sword & Shield,1,weapon,24,Ruffian,mig 1,att 3,None,None
1,434,146.0,Brigandine,1,armor,24,Burglar,agi 1,dmg 1,None,None
8,441,150.0,Twin Daggers,1,weapon,24,Scoundrel,cun 1,att 2,None,None
9,442,150.0,Vestment,1,armor,24,Scoundrel,cun 1,att 2,None,None
6,439,149.0,Blackthorn,1,weapon,24,Burglar,agi 1,dmg 1,None,None
5,438,148.0,Chain Maille,1,armor,24,Ruffian,mig 1,att 3,None,None
4,437,148.0,Sword & Shield,1,weapon,24,Ruffian,mig 1,att 3,None,None
